You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Title: The Future of Artificial Intelligence: Trends, Players, and News

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Purpose of the article

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI ethics and bias considerations
- Automation and robotics integration

III. Key Players in the AI Industry
- Google
- Amazon
- Microsoft
- IBM
- Tesla

IV. Noteworthy New

I now can give a great answer.

Final Answer:
# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a transformative force in various industries, revolutionizing the way we work, communicate, and interact with technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in driving innovation and efficiency across sectors. This article aims to delve into the latest trends, key players, and noteworthy news in the AI industry, providing valuable insights for tech-savvy individuals, business professionals, students, researchers, and academics interested in the future of AI.

## Latest Trends in Artificial Intelligence
The field of AI is constantly evolving, with rapid advancements in machine learning, natural language processing, ethics, bias considerations, and automation. Machine learning algorithms have become more sophisticated, enabling machines to learn from data and ma

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a transformative force in various industries, revolutionizing the way we work, communicate, and interact with technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in driving innovation and efficiency across sectors. This article aims to delve into the latest trends, key players, and noteworthy news in the AI industry, providing valuable insights for tech-savvy individuals, business professionals, students, researchers, and academics interested in the future of AI.

## Latest Trends in Artificial Intelligence
The field of AI is constantly evolving, with rapid advancements in machine learning, natural language processing, ethics, bias considerations, and automation. Machine learning algorithms have become more sophisticated, enabling machines to learn from data and make decisions without explicit programming. Natural language processing technologies have improved communication between humans and machines, enhancing voice recognition and language translation capabilities. Ethical concerns regarding bias in AI algorithms have also gained prominence, urging developers to prioritize fairness and transparency in AI systems. Automation and robotics integration have streamlined processes in industries such as manufacturing, healthcare, and finance, increasing productivity and reducing human error.

## Key Players in the AI Industry
Several tech giants have emerged as leaders in the AI industry, driving innovation and shaping the future of AI technologies. Companies like Google, Amazon, Microsoft, IBM, and Tesla have invested heavily in AI research and development, creating cutting-edge solutions for businesses and consumers. Google's DeepMind project has made significant strides in AI research, while Amazon's Alexa and Microsoft's Cortana have revolutionized virtual assistant technologies. IBM's Watson platform has paved the way for AI applications in healthcare and finance, while Tesla's self-driving cars represent the future of autonomous vehicles powered by AI.

## Noteworthy News in Artificial Intelligence
Recent breakthroughs in AI technology have captured headlines, showcasing the potential of AI to transform industries and improve our daily lives. From healthcare diagnostics to financial forecasting, AI has demonstrated its ability to drive innovation and efficiency. However, concerns about the impact of AI on job markets have also surfaced, raising questions about the future of work in an AI-driven world. Ethical considerations in AI development, such as data privacy, algorithmic bias, and autonomous decision-making, have prompted discussions about the responsible use of AI technologies. Future predictions for AI adoption suggest a continued growth in AI applications across industries, presenting both opportunities and challenges for businesses and society.

In conclusion, the future of Artificial Intelligence is filled with endless possibilities and challenges, as we navigate the complex landscape of AI technologies and their implications. By staying informed about the latest trends, key players, and noteworthy news in the AI industry, individuals and organizations can make informed decisions about the role of AI in shaping our collective future. As AI continues to evolve and transform the way we live and work, it is essential to engage in discussions, stay updated on developments, and explore opportunities to deepen our knowledge and understanding of this groundbreaking technology.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
topic = "GEN AI"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on GEN AI.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: GEN AI

Introduction:
- Brief overview of GEN AI and its impact on various industries
- Mention of the latest trends and key players in the GEN AI space
- Preview of the key points to be discussed in the article

Key Points:
1. Latest Trends in GEN AI:
- Advancements in machine learning algorithms
- Integration of AI with IoT devices
- Focus on ethical AI development

2. Key Players in the GEN AI Industry:
- Google DeepMind
- OpenAI
- IBM Watson

3. Noteworthy News in GEN AI:
- Recent partnerships between tech c

I now can give a great answer

Final Answer: 

# The Rise of GEN AI: Latest Trends and Key Players in the Industry

## Introduction
The rise of General Artificial Intelligence (GEN AI) is reshaping industries worldwide with its innovative capabilities and potential. From enhanced machine learning algorithms to the seamless integration of AI with IoT devices, the impact of GEN AI is unmistakable. Leading the charge in this transformative space are key players such as Google DeepMind, OpenAI, and IBM Watson, who are shaping the landscape of artificial intelligence. This article aims to explore the latest trends, key players, and noteworthy news in the GEN AI industry, offering valuable insights for technology enthusiasts, business professionals, and researchers.

## Latest Trends in GEN AI
The evolution of GEN AI is being primarily propelled by advancements in machine learning algorithms, enabling the development of more sophisticated and efficient AI systems. The fusion of AI with IoT d

In [15]:
Markdown(result)

# The Rise of GEN AI: Latest Trends and Key Players in the Industry

## Introduction
The rise of General Artificial Intelligence (GEN AI) is reshaping industries worldwide with its innovative capabilities and potential. From enhanced machine learning algorithms to the seamless integration of AI with IoT devices, the impact of GEN AI is unmistakable. Leading the charge in this transformative space are key players such as Google DeepMind, OpenAI, and IBM Watson, who are shaping the landscape of artificial intelligence. This article aims to explore the latest trends, key players, and noteworthy news in the GEN AI industry, offering valuable insights for technology enthusiasts, business professionals, and researchers.

## Latest Trends in GEN AI
The evolution of GEN AI is being primarily propelled by advancements in machine learning algorithms, enabling the development of more sophisticated and efficient AI systems. The fusion of AI with IoT devices is fostering a network of interconnected devices capable of communication and mutual learning. Additionally, there is a growing emphasis on ethical AI development to ensure responsible design and deployment of AI systems, with a focus on considering the societal impact.

## Key Players in the GEN AI Industry
Google DeepMind, renowned for its groundbreaking AI research, stands at the forefront of GEN AI development, driving innovation in the field. OpenAI, established with the goal of ensuring that artificial general intelligence benefits humanity as a whole, is pushing the boundaries of AI capabilities. IBM Watson, through its cognitive computing platform, is empowering businesses to utilize AI for strategic decision-making and innovation.

## Noteworthy News in GEN AI
Recent collaborations among tech companies for AI research are fostering a culture of collaboration and innovation within the industry. Discussions surrounding the impact of GEN AI on job automation and workforce transformation are prevalent, sparking conversations about the future of work in an AI-driven world. Regulatory advancements in AI ethics and data privacy are guiding the development and deployment of AI systems with a focus on responsible practices.

## Conclusion
As GEN AI continues to advance and revolutionize industries, staying informed about the latest trends and key players is crucial. With an emphasis on ethical AI development and responsible deployment, the potential for positive transformation in the field of artificial intelligence is substantial. Whether you are a technology enthusiast, a business professional, or a researcher in the AI realm, exploring the realm of GEN AI will provide valuable insights and growth opportunities.

In conclusion, the rapid evolution of the GEN AI industry is fueled by key players driving innovation and collaboration. By staying abreast of the latest trends and advancements, individuals and organizations can harness the power of artificial intelligence for business growth and societal impact. Embracing the future of AI technology and exploring its diverse applications will pave the way for a more intelligent and interconnected world.

*Call to action: Delve deeper into the world of GEN AI by immersing yourself in resources from reputable sources and remaining engaged with the latest news and breakthroughs in artificial intelligence.*

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).